<a href="https://colab.research.google.com/github/Bzhamdi/holbertonschool-machine_learning/blob/main/reinforcement_learning/0x01-deep_q_learning/q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [2]:
import gym 
import random

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

In [53]:
# create Environment with OpenAI Gym
import numpy as np


env = gym.make('BreakoutDeterministic-v4')
height, width, channels = env.observation_space.shape
np.random.seed(123)
env.seed(123)
actions = env.action_space.n

In [55]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [56]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D,Activation
from tensorflow.keras.optimizers import Adam

In [57]:
#Create a Deep Learning Model with Keras
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(actions))
    model.add(Activation('linear'))
    return model

In [58]:

model = build_model(height, width, channels, actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
activation_12 (Activation)   (None, 3, 24, 18, 64)     0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
activation_13 (Activation)   (None, 3, 22, 16, 64)     0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)              

In [59]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [61]:
# Build Agent with Keras-RL
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [62]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [64]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 20 steps ...
done, took 0.884 seconds


In [ ]:

scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [32]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [66]:
!pip install colabgymrender
from colabgymrender.recorder import Recorder


directory = './video'
env = Recorder(env, directory)

observation = env.reset()
terminal = False
while not terminal:
  action = env.action_space.sample()
  observation, reward, terminal, info = env.step(action)

env.play()

dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')